In [4]:
! pip install requests pydantic pandas python-dotenv

In [54]:
from pydantic import BaseModel, ValidationError
import requests
import pandas as pd
from typing import List, Optional, Union, Dict, Any
from dotenv import load_dotenv
import os
from datetime import datetime
from uuid import UUID


In [58]:
# Example models based on the hypothetical data structure from the API

class Customer(BaseModel):
    name: str
    custom_fields: Dict[str, Any]
    external_id: Optional[str]
    ingest_aliases: List[str]
    id: str
    customer_config: Dict[str, Any]

# Found on LineItem Model
class CreditType(BaseModel):
    id: UUID
    name: str

# Sublines on LineItem Model
class SubLineItem(BaseModel):
    charge_id: UUID
    name: str
    subtotal: float
    price: float
    quantity: int
    custom_fields: Dict[str, str]


class LineItem(BaseModel):
    total: float
    credit_type: CreditType
    name: str
    product_id: UUID
    quantity: int
    custom_fields: Dict[str, str]
    sub_line_items: List[SubLineItem]

class InvoiceAdjustment(BaseModel):
    total: float
    credit_type: CreditType

class Invoice(BaseModel):
    id: UUID
    start_timestamp: datetime
    end_timestamp: datetime
    customer_id: UUID
    customer_custom_fields: Dict[str, str]
    type: str
    credit_type: CreditType
    plan_id: UUID
    plan_name: str
    plan_custom_fields: Dict[str, str]
    status: str
    total: float
    external_invoice: Optional[str]
    subtotal: float
    line_items: List[LineItem]
    invoice_adjustments: List[InvoiceAdjustment]
    custom_fields: Dict[str, str]
    billable_status: str

# Todo: Implement the API client deal with data wrapper this way
class ApiResponse(BaseModel):
    data: List[DataItem]
    next_page: Optional[str] = None



In [87]:
load_dotenv()
API_KEY = os.getenv("API_KEY")
BASE_URL = os.getenv("BASE_URL")

def _request(endpoint: str, params:dict = {}) -> Dict[str, Any]:
    headers = {"Authorization": f"Bearer {API_KEY}"}
    full_endpoint = f"{BASE_URL}/{endpoint}"
    print(full_endpoint)
    response = requests.get(f"{full_endpoint}", headers=headers, params=params)
    return response.json()

def get_customers(**params) -> List[Customer]:
        raw_data = _request("customers", params=params)
        print(params)
        try:
            # Extract customers from 'data' key and convert each entry to a Customer model
            return [Customer(**item) for item in raw_data.get("data", [])]
        except ValidationError as e:
            print("Validation error:", e)
            return []
        
def get_customer(customer_id: str) -> Customer:
    raw_data = _request(f"customers/{customer_id}").get("data", {})
    try:
        return Customer(**raw_data)
    except ValidationError as e:
        print("Validation error:", e)
        return None
    

def get_customer_invoices(customer_id: str) -> List[Invoice]:
    raw_data = _request(f"customers/{customer_id}/invoices").get("data", [])
    try:
        #return raw_data
        return [Invoice(**item) for item in raw_data]
    except ValidationError as e:
        print("Validation error:", e)
        return []
    
"""
How to get customer balances
response = requests.post(
    "https://api.metronome.com/v1/contracts/customerBalances/list",
    headers={
        "Authorization": f"Bearer {token}",
    },
    json={
        "customer_id": "13117714-3f05-48e5-a6e9-a66093f13b4d",
        "id": "6162d87b-e5db-4a33-b7f2-76ce6ead4e85",
        "include_ledgers": True,
    },
)

data = response.json()"""

def get_balances(**params) -> List[Dict[str, Any]]:
    raw_data = _request("contracts/customerBalances/list", params=params)
    return raw_data

In [74]:
customers = get_customers(limit=10)
# Convert the pydantic list of customers back to python
customers_list_dict = [customer.dict() for customer in customers]
# Convert the list of dictionaries to a pandas DataFrame
df = pd.DataFrame(customers_list_dict)
df.head()

https://api.metronome.com/v1/customers
{'limit': 10}


,name,custom_fields,external_id,ingest_aliases,id,customer_config
0,DJLAPQ Ltd.,{},ERTMPA4M,[ERTMPA4M],004747b8-9124-4060-989a-8d1075af2424,{'salesforce_account_id': None}
1,OUWIIM Inc.,{},5O9CTU3A,[5O9CTU3A],0602ebf7-659e-470a-a536-9fbd413fb42b,{'salesforce_account_id': None}
2,C3 Company,{},me@nora.edu,[me@nora.edu],12184764-5687-4690-8794-35efc5586e72,{'salesforce_account_id': None}
3,RTLKR LLC,{},3DWGSEDM,[3DWGSEDM],15b367c9-04b9-4064-9a58-b589928898fd,{'salesforce_account_id': None}
4,BQYGSA Corp.,{},L9RN960G,[L9RN960G],1715df37-9b9b-4829-b381-e7febaefb102,{'salesforce_account_id': None}


In [75]:
len(customers)

10

In [61]:
# Get a single customer
#customer = get_customer(customers[0].id)

# Get invoices for a customer
invoices = get_customer_invoices(customers[0].id)

https://api.metronome.com/v1/customers/004747b8-9124-4060-989a-8d1075af2424/invoices


In [100]:
# Convert the pydantic list of invoices back to python dicts
invoices_list_dict = [invoice.dict() for invoice in invoices]

# Unnest the python dicts
def unnest_dict(d, parent_key='', sep='_'):
    items = []
    for k, v in d.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            items.extend(unnest_dict(v, new_key, sep=sep).items())
        elif isinstance(v, list):
            for i, item in enumerate(v):
                if isinstance(item, dict):
                    items.extend(unnest_dict(item, f"{new_key}{sep}{i}", sep=sep).items())
                else:
                    items.append((f"{new_key}{sep}{i}", item))
        else:
            items.append((new_key, v))
    return dict(items)

# Unnest each invoice dict
unnested_invoices = [unnest_dict(invoice) for invoice in invoices_list_dict]

# Convert the list of unnested dictionaries to a pandas DataFrame
df_invoices = pd.DataFrame(unnested_invoices)
df_invoices.head()

,id,start_timestamp,end_timestamp,customer_id,type,credit_type_id,credit_type_name,plan_id,plan_name,plan_custom_fields_x-seed-obj-id,...,line_items_2_sub_line_items_1_quantity,line_items_2_sub_line_items_2_charge_id,line_items_2_sub_line_items_2_name,line_items_2_sub_line_items_2_subtotal,line_items_2_sub_line_items_2_price,line_items_2_sub_line_items_2_quantity,billable_status,invoice_adjustments_0_total,invoice_adjustments_0_credit_type_id,invoice_adjustments_0_credit_type_name
0,bad1671b-f710-40c8-ad5a-02f4ab1175a3,2024-04-01 00:00:00+00:00,2024-05-01 00:00:00+00:00,004747b8-9124-4060-989a-8d1075af2424,PLAN_ARREARS,2714e483-4ff1-48e4-9e25-ac732e8f24f2,USD (cents),5b81cb83-fa0b-4ad2-94c7-bbd81502e2d9,Free Plan,free,...,0,7657fb9f-d17a-419a-b87f-83a696d47b5e,Images (1024x1024),0.0,45.0,0,billable,NaN,NaN,NaN
1,996a662a-ca7c-4e1e-86bb-237ccec79153,2024-05-01 00:00:00+00:00,2024-06-01 00:00:00+00:00,004747b8-9124-4060-989a-8d1075af2424,PLAN_ARREARS,2714e483-4ff1-48e4-9e25-ac732e8f24f2,USD (cents),5b81cb83-fa0b-4ad2-94c7-bbd81502e2d9,Free Plan,free,...,0,7657fb9f-d17a-419a-b87f-83a696d47b5e,Images (1024x1024),0.0,45.0,0,billable,NaN,NaN,NaN
2,29658a41-6eff-4515-aaec-cae015ad6126,2024-06-01 00:00:00+00:00,2024-07-01 00:00:00+00:00,004747b8-9124-4060-989a-8d1075af2424,PLAN_ARREARS,2714e483-4ff1-48e4-9e25-ac732e8f24f2,USD (cents),5b81cb83-fa0b-4ad2-94c7-bbd81502e2d9,Free Plan,free,...,0,7657fb9f-d17a-419a-b87f-83a696d47b5e,Images (1024x1024),0.0,45.0,0,billable,-17369.85,2714e483-4ff1-48e4-9e25-ac732e8f24f2,USD (cents)
3,672b212b-5a87-477d-b85d-b5f957ff75d1,2024-07-01 00:00:00+00:00,2024-08-01 00:00:00+00:00,004747b8-9124-4060-989a-8d1075af2424,PLAN_ARREARS,2714e483-4ff1-48e4-9e25-ac732e8f24f2,USD (cents),5b81cb83-fa0b-4ad2-94c7-bbd81502e2d9,Free Plan,free,...,2203,7657fb9f-d17a-419a-b87f-83a696d47b5e,Images (1024x1024),103230.0,45.0,2294,billable,-82630.15,2714e483-4ff1-48e4-9e25-ac732e8f24f2,USD (cents)
4,513e9159-0931-51f5-81e1-a7b0d6de6dbf,2024-08-01 00:00:00+00:00,2024-09-01 00:00:00+00:00,004747b8-9124-4060-989a-8d1075af2424,PLAN_ARREARS,2714e483-4ff1-48e4-9e25-ac732e8f24f2,USD (cents),5b81cb83-fa0b-4ad2-94c7-bbd81502e2d9,Free Plan,free,...,842,7657fb9f-d17a-419a-b87f-83a696d47b5e,Images (1024x1024),30510.0,45.0,678,billable,NaN,NaN,NaN


In [91]:
# Get balances for a customer
balances = [get_balances(customer_id=customers[x].id) for x in range(0, len(customers))]

https://api.metronome.com/v1/contracts/customerBalances/list
https://api.metronome.com/v1/contracts/customerBalances/list
https://api.metronome.com/v1/contracts/customerBalances/list
https://api.metronome.com/v1/contracts/customerBalances/list
https://api.metronome.com/v1/contracts/customerBalances/list
https://api.metronome.com/v1/contracts/customerBalances/list
https://api.metronome.com/v1/contracts/customerBalances/list
https://api.metronome.com/v1/contracts/customerBalances/list
https://api.metronome.com/v1/contracts/customerBalances/list
https://api.metronome.com/v1/contracts/customerBalances/list


In [96]:
_request("https://api.metronome.com/v1/contracts/customerCredits/list", params={"customer_id":customers[0].id})

https://api.metronome.com/v1/https://api.metronome.com/v1/contracts/customerCredits/list


{'message': 'Resource not found'}

In [97]:
response = requests.post(
    "https://api.metronome.com/v1/contracts/customerCredits/list",
    headers={
        "Authorization": f"Bearer {API_KEY}",
    },
    json={
        "customer_id": "13117714-3f05-48e5-a6e9-a66093f13b4d",
        "credit_id": "6162d87b-e5db-4a33-b7f2-76ce6ead4e85",
        "include_ledgers": True,
    },
)

In [98]:
response

<Response [404]>

In [101]:
customers[0].id 

'004747b8-9124-4060-989a-8d1075af2424'

In [102]:
invoices[0].customer_id

UUID('004747b8-9124-4060-989a-8d1075af2424')

In [107]:
customers_df_check = pd.read_csv("customers.csv")
print(customers_df_check.id)

invoices_df_check = pd.read_csv("invoices.csv")
print(invoices_df_check.customer_id)

0    004747b8-9124-4060-989a-8d1075af2424
Name: id, dtype: object
0    004747b8-9124-4060-989a-8d1075af2424
1    004747b8-9124-4060-989a-8d1075af2424
2    004747b8-9124-4060-989a-8d1075af2424
3    004747b8-9124-4060-989a-8d1075af2424
4    004747b8-9124-4060-989a-8d1075af2424
5    004747b8-9124-4060-989a-8d1075af2424
6    004747b8-9124-4060-989a-8d1075af2424
7    004747b8-9124-4060-989a-8d1075af2424
Name: customer_id, dtype: object


In [108]:
pd.merge(customers_df_check, invoices_df_check, left_on="id", right_on="customer_id", how="left")

,name,external_id,ingest_aliases_0,id_x,customer_config_salesforce_account_id,id_y,start_timestamp,end_timestamp,customer_id,type,...,line_items_2_sub_line_items_1_quantity,line_items_2_sub_line_items_2_charge_id,line_items_2_sub_line_items_2_name,line_items_2_sub_line_items_2_subtotal,line_items_2_sub_line_items_2_price,line_items_2_sub_line_items_2_quantity,billable_status,invoice_adjustments_0_total,invoice_adjustments_0_credit_type_id,invoice_adjustments_0_credit_type_name
0,DJLAPQ Ltd.,ERTMPA4M,ERTMPA4M,004747b8-9124-4060-989a-8d1075af2424,NaN,bad1671b-f710-40c8-ad5a-02f4ab1175a3,2024-04-01 00:00:00+00:00,2024-05-01 00:00:00+00:00,004747b8-9124-4060-989a-8d1075af2424,PLAN_ARREARS,...,0,7657fb9f-d17a-419a-b87f-83a696d47b5e,Images (1024x1024),0.0,45.0,0,billable,NaN,NaN,NaN
1,DJLAPQ Ltd.,ERTMPA4M,ERTMPA4M,004747b8-9124-4060-989a-8d1075af2424,NaN,996a662a-ca7c-4e1e-86bb-237ccec79153,2024-05-01 00:00:00+00:00,2024-06-01 00:00:00+00:00,004747b8-9124-4060-989a-8d1075af2424,PLAN_ARREARS,...,0,7657fb9f-d17a-419a-b87f-83a696d47b5e,Images (1024x1024),0.0,45.0,0,billable,NaN,NaN,NaN
2,DJLAPQ Ltd.,ERTMPA4M,ERTMPA4M,004747b8-9124-4060-989a-8d1075af2424,NaN,29658a41-6eff-4515-aaec-cae015ad6126,2024-06-01 00:00:00+00:00,2024-07-01 00:00:00+00:00,004747b8-9124-4060-989a-8d1075af2424,PLAN_ARREARS,...,0,7657fb9f-d17a-419a-b87f-83a696d47b5e,Images (1024x1024),0.0,45.0,0,billable,-17369.85,2714e483-4ff1-48e4-9e25-ac732e8f24f2,USD (cents)
3,DJLAPQ Ltd.,ERTMPA4M,ERTMPA4M,004747b8-9124-4060-989a-8d1075af2424,NaN,672b212b-5a87-477d-b85d-b5f957ff75d1,2024-07-01 00:00:00+00:00,2024-08-01 00:00:00+00:00,004747b8-9124-4060-989a-8d1075af2424,PLAN_ARREARS,...,2203,7657fb9f-d17a-419a-b87f-83a696d47b5e,Images (1024x1024),103230.0,45.0,2294,billable,-82630.15,2714e483-4ff1-48e4-9e25-ac732e8f24f2,USD (cents)
4,DJLAPQ Ltd.,ERTMPA4M,ERTMPA4M,004747b8-9124-4060-989a-8d1075af2424,NaN,513e9159-0931-51f5-81e1-a7b0d6de6dbf,2024-08-01 00:00:00+00:00,2024-09-01 00:00:00+00:00,004747b8-9124-4060-989a-8d1075af2424,PLAN_ARREARS,...,842,7657fb9f-d17a-419a-b87f-83a696d47b5e,Images (1024x1024),30510.0,45.0,678,billable,NaN,NaN,NaN
5,DJLAPQ Ltd.,ERTMPA4M,ERTMPA4M,004747b8-9124-4060-989a-8d1075af2424,NaN,86c5044b-79a1-5a65-8fa7-917ecad967e2,2024-09-01 00:00:00+00:00,2024-10-01 00:00:00+00:00,004747b8-9124-4060-989a-8d1075af2424,PLAN_ARREARS,...,0,7657fb9f-d17a-419a-b87f-83a696d47b5e,Images (1024x1024),0.0,45.0,0,billable,NaN,NaN,NaN
6,DJLAPQ Ltd.,ERTMPA4M,ERTMPA4M,004747b8-9124-4060-989a-8d1075af2424,NaN,07a72d98-deeb-5150-9912-8a589f04d001,2024-10-01 00:00:00+00:00,2024-11-01 00:00:00+00:00,004747b8-9124-4060-989a-8d1075af2424,PLAN_ARREARS,...,0,7657fb9f-d17a-419a-b87f-83a696d47b5e,Images (1024x1024),0.0,45.0,0,billable,NaN,NaN,NaN
7,DJLAPQ Ltd.,ERTMPA4M,ERTMPA4M,004747b8-9124-4060-989a-8d1075af2424,NaN,b645a329-a728-5c0b-8055-3eb7458adff2,2024-11-01 00:00:00+00:00,2024-12-01 00:00:00+00:00,004747b8-9124-4060-989a-8d1075af2424,PLAN_ARREARS,...,0,7657fb9f-d17a-419a-b87f-83a696d47b5e,Images (1024x1024),0.0,45.0,0,billable,NaN,NaN,NaN


In [109]:
# Summary reporting
# Rollup the invoice data to the customer level
# Group by customer_id and sum the total and subtotal
customer_summary = df_invoices.groupby("customer_id")[["total", "subtotal"]].sum()

In [110]:
customer_summary

,total,subtotal
customer_id,,
004747b8-9124-4060-989a-8d1075af2424,328068.6,428068.6


In [116]:
invoices_df_check[invoices_df_check.customer_id == customers[0].id]

,id,start_timestamp,end_timestamp,customer_id,type,credit_type_id,credit_type_name,plan_id,plan_name,plan_custom_fields_x-seed-obj-id,...,line_items_2_sub_line_items_1_quantity,line_items_2_sub_line_items_2_charge_id,line_items_2_sub_line_items_2_name,line_items_2_sub_line_items_2_subtotal,line_items_2_sub_line_items_2_price,line_items_2_sub_line_items_2_quantity,billable_status,invoice_adjustments_0_total,invoice_adjustments_0_credit_type_id,invoice_adjustments_0_credit_type_name
0,bad1671b-f710-40c8-ad5a-02f4ab1175a3,2024-04-01 00:00:00+00:00,2024-05-01 00:00:00+00:00,004747b8-9124-4060-989a-8d1075af2424,PLAN_ARREARS,2714e483-4ff1-48e4-9e25-ac732e8f24f2,USD (cents),5b81cb83-fa0b-4ad2-94c7-bbd81502e2d9,Free Plan,free,...,0,7657fb9f-d17a-419a-b87f-83a696d47b5e,Images (1024x1024),0.0,45.0,0,billable,NaN,NaN,NaN
1,996a662a-ca7c-4e1e-86bb-237ccec79153,2024-05-01 00:00:00+00:00,2024-06-01 00:00:00+00:00,004747b8-9124-4060-989a-8d1075af2424,PLAN_ARREARS,2714e483-4ff1-48e4-9e25-ac732e8f24f2,USD (cents),5b81cb83-fa0b-4ad2-94c7-bbd81502e2d9,Free Plan,free,...,0,7657fb9f-d17a-419a-b87f-83a696d47b5e,Images (1024x1024),0.0,45.0,0,billable,NaN,NaN,NaN
2,29658a41-6eff-4515-aaec-cae015ad6126,2024-06-01 00:00:00+00:00,2024-07-01 00:00:00+00:00,004747b8-9124-4060-989a-8d1075af2424,PLAN_ARREARS,2714e483-4ff1-48e4-9e25-ac732e8f24f2,USD (cents),5b81cb83-fa0b-4ad2-94c7-bbd81502e2d9,Free Plan,free,...,0,7657fb9f-d17a-419a-b87f-83a696d47b5e,Images (1024x1024),0.0,45.0,0,billable,-17369.85,2714e483-4ff1-48e4-9e25-ac732e8f24f2,USD (cents)
3,672b212b-5a87-477d-b85d-b5f957ff75d1,2024-07-01 00:00:00+00:00,2024-08-01 00:00:00+00:00,004747b8-9124-4060-989a-8d1075af2424,PLAN_ARREARS,2714e483-4ff1-48e4-9e25-ac732e8f24f2,USD (cents),5b81cb83-fa0b-4ad2-94c7-bbd81502e2d9,Free Plan,free,...,2203,7657fb9f-d17a-419a-b87f-83a696d47b5e,Images (1024x1024),103230.0,45.0,2294,billable,-82630.15,2714e483-4ff1-48e4-9e25-ac732e8f24f2,USD (cents)
4,513e9159-0931-51f5-81e1-a7b0d6de6dbf,2024-08-01 00:00:00+00:00,2024-09-01 00:00:00+00:00,004747b8-9124-4060-989a-8d1075af2424,PLAN_ARREARS,2714e483-4ff1-48e4-9e25-ac732e8f24f2,USD (cents),5b81cb83-fa0b-4ad2-94c7-bbd81502e2d9,Free Plan,free,...,842,7657fb9f-d17a-419a-b87f-83a696d47b5e,Images (1024x1024),30510.0,45.0,678,billable,NaN,NaN,NaN
5,86c5044b-79a1-5a65-8fa7-917ecad967e2,2024-09-01 00:00:00+00:00,2024-10-01 00:00:00+00:00,004747b8-9124-4060-989a-8d1075af2424,PLAN_ARREARS,2714e483-4ff1-48e4-9e25-ac732e8f24f2,USD (cents),5b81cb83-fa0b-4ad2-94c7-bbd81502e2d9,Free Plan,free,...,0,7657fb9f-d17a-419a-b87f-83a696d47b5e,Images (1024x1024),0.0,45.0,0,billable,NaN,NaN,NaN
6,07a72d98-deeb-5150-9912-8a589f04d001,2024-10-01 00:00:00+00:00,2024-11-01 00:00:00+00:00,004747b8-9124-4060-989a-8d1075af2424,PLAN_ARREARS,2714e483-4ff1-48e4-9e25-ac732e8f24f2,USD (cents),5b81cb83-fa0b-4ad2-94c7-bbd81502e2d9,Free Plan,free,...,0,7657fb9f-d17a-419a-b87f-83a696d47b5e,Images (1024x1024),0.0,45.0,0,billable,NaN,NaN,NaN
7,b645a329-a728-5c0b-8055-3eb7458adff2,2024-11-01 00:00:00+00:00,2024-12-01 00:00:00+00:00,004747b8-9124-4060-989a-8d1075af2424,PLAN_ARREARS,2714e483-4ff1-48e4-9e25-ac732e8f24f2,USD (cents),5b81cb83-fa0b-4ad2-94c7-bbd81502e2d9,Free Plan,free,...,0,7657fb9f-d17a-419a-b87f-83a696d47b5e,Images (1024x1024),0.0,45.0,0,billable,NaN,NaN,NaN


In [ ]:
def load_and_process_data(api_results,json_file_raw,json_file_flat, csv_file):
    # Convert data models to dictionaries
    data_dicts = models_to_dicts(api_results)
    print(data_dicts)

    # Write JSON data to JSON file
    with open(json_file_raw, "w") as f:
        json.dump(data_dicts, f)

    # Flatten the JSON data
    flat_data = unnest_dict(data_dicts)

    # Write flattened JSON data to JSON file
    with open(json_file_flat, "w") as f:
        json.dump(flat_data, f)
    # Convert JSON data to a Pandas DataFrame and write to CSV
    df = pd.DataFrame(flat_data)
    df.to_csv(csv_file, index=False)
    return df